In [12]:
import re
from matplotlib import pyplot as plt
import math
import random

customer_arrivals=[]
interarrivals=[]
analysis=[]
validation=[]

def interarrival_times():
    with open('customer_arrivals.csv','r') as cusfile:
        for line in cusfile:
            x = re.split('\n+',line.rstrip())
            customer_arrivals.append(x)
        for i in range(len(customer_arrivals)-1):
            interarrivals.append(float(customer_arrivals[i+1][0]) - float(customer_arrivals[i][0]))
        ilen=round(len(interarrivals)/4)
        for i in range(ilen*3):
            analysis.append(interarrivals[i])
        for i in interarrivals[(ilen*3):]:
            validation.append(i)

In [13]:
interarrival_times()
#First we need to calculate our lambda parameter with Maximum Likelihood Estimation.
lambdaparameter = len(analysis)/sum(analysis)
#Now we need to take the inverse of the CDF of Exponential Distribution (1-e^-lambdaparameter*x)
def inverse(lambdaparameter,x):
    return -math.log(1-x)/lambdaparameter
#And pass a float from uniform distribution between 0,1 to this inverse function.
def exp_random_generate(lambdaparameter):
    x = random.random()
    return inverse(lambdaparameter,x)

In [14]:
def fel_key(event):
    return event['time']

In [15]:
def linear_random_cab_reorder(seed):
    return ((31*seed+7)%15)+1

In [16]:
def linear_random_cab_expire(seed):
    return ((11*seed+7)%5)+7

In [20]:
#Ipek Erdogan,150130102
#We have 7 functions for all of the events. 3 of them are for cabbages, 4 of them are for customers.
#When a cabbage arrives, an event for checking if it's expired or not, is added to the future event list. There is always a contol
#phase for every cabbage. Since we hold cabbages in a stock list in ascending order of their expire time, we can check
#if they're expired or already gone, with comparison of clock and the first element of the stock list.
def process_cabbages_arrival_event(state,fel,clock,stock):
    seed = random.uniform(0,100)
    time = clock+linear_random_cab_expire(seed)
    #I replaced the uniform random generation between 7 and 12 above (random.uniform(7.0, 12.0)) with my exponential dist. random generator.
    fel.append({'type':'check_if_cabbage_expired','time': time})
    stock.append(time)
    state['stocked_cabbages']+=1
    stock.sort()

def process_cabbage_reorder_event(state,fel,clock):
    fel.append({'type':'cabbage_arrival','time':clock})
    
def process_cabbage_expired_event(state,fel,clock,stock):
    if(len(stock) != 0):
        if (stock[0] == clock):
            print("CABBAGE HAS EXPIRED.")
            state['expired_cabbages']+=1
            del stock[0]
            state['stocked_cabbages']-=1
            seed = random.uniform(0,100)
            fel.append({'type':'cabbage_reorder','time':clock+linear_random_cab_reorder(seed)}) 
            #I replaced the uniform random generation between 1 and 15 above (random.uniform(1.0, 15.0)) with my custom linear congruential random generator.
        else:
            return
    else:
        return
    
def process_customer_leaving_happy_event(state):
    state['happy_customer']+=1
    
def process_customer_leaving_sad_event(state):
    state['sad_customer']+=1
    
def process_customer_arrival_event(state,fel,clock):
    fel.append({'type':'customer_arrival','time':clock+exp_random_generate(lambdaparameter)})
    #I replaced the uniform random generation between 0 and 3 above (random.uniform(0.0, 3.0)) with my exponential dist. random generator.
    fel.append({'type':'customer_purchase','time':clock})
    state['customer']+=1
    
def process_customer_purchase_event(state,fel,clock,stock):
    if (state['stocked_cabbages'] == 0):
        process_customer_leaving_sad_event(state)
        fel.append({'type':'customer_leaves_sad','time':clock})
    else:
        del stock[0]
        state['stocked_cabbages']-=1
        process_customer_leaving_happy_event(state)
        fel.append({'type':'customer_leaves_happy','time':clock})
        seed = random.uniform(0,100)
        fel.append({'type':'cabbage_reorder','time':clock+linear_random_cab_reorder(seed)})
        #I replaced the uniform random generation between 1 and 15 above (random.uniform(1.0, 15.0)) with my custom linear congruential random generator.


In [21]:
clock = 0
fel = []
stock = []
state = {'stocked_cabbages':13,'happy_customer':0,'sad_customer':0, 'customer':0, 'expired_cabbages':0}
#All these implementations below are for initialization of our simulation.
first_event = {'type':'cabbage_arrival','time':0}
#Our range in here is our initial stock, you can change it and see the sad/happy customer results for other values.
for i in range(13):
    seed = random.uniform(0,100)
    time = linear_random_cab_expire(seed)
    #I replaced the uniform random generation between 7 and 12 above (random.uniform(7.0, 12.0)) with my exponential dist. random generator.
    fel.append({'type':'check_if_cabbage_expired','time': time})
    stock.append(time)
stock.sort()
second_event = {'type':'customer_arrival','time':exp_random_generate(lambdaparameter)}
#I replaced the uniform random generation between 0 and 3 above (random.uniform(0.0, 3.0)) with my exponential dist. random generator.
fel.append(second_event)

In [22]:
while len(fel)!=0 and state['customer']<100:
    fel.sort(key=fel_key)
    next_event = fel[0]
    del fel[0]
    print("processing event",next_event)
    clock = next_event['time']
    if next_event['type'] == 'cabbage_arrival':
        process_cabbages_arrival_event(state,fel,clock,stock)
    elif next_event['type'] == 'customer_arrival':
        process_customer_arrival_event(state,fel,clock)
    elif next_event['type'] == 'customer_purchase':
        process_customer_purchase_event(state,fel,clock,stock)
    elif next_event['type'] == 'cabbage_reorder':
        process_cabbage_reorder_event(state,fel,clock)
    elif next_event['type'] == 'check_if_cabbage_expired':
        process_cabbage_expired_event(state,fel,clock,stock)
                                        
#You can uncomment rows below and comment the other print functions in the code to see the results according to the different
#stock or customer values.
#print(state['customer'])
#print(state['happy_customer'])
#print(state['sad_customer'])
#print(state['expired_cabbages'])

processing event {'type': 'check_if_cabbage_expired', 'time': 5.298087308621916}
CABBAGE HAS EXPIRED.
processing event {'type': 'check_if_cabbage_expired', 'time': 5.393669877839329}
CABBAGE HAS EXPIRED.
processing event {'type': 'check_if_cabbage_expired', 'time': 5.842161814870451}
CABBAGE HAS EXPIRED.
processing event {'type': 'check_if_cabbage_expired', 'time': 6.83998761507894}
CABBAGE HAS EXPIRED.
processing event {'type': 'check_if_cabbage_expired', 'time': 6.962973874657905}
CABBAGE HAS EXPIRED.
processing event {'type': 'customer_arrival', 'time': 7.106436205127664}
processing event {'type': 'customer_purchase', 'time': 7.106436205127664}
processing event {'type': 'customer_leaves_happy', 'time': 7.106436205127664}
processing event {'type': 'check_if_cabbage_expired', 'time': 7.50304290089602}
processing event {'type': 'check_if_cabbage_expired', 'time': 8.069239050739952}
CABBAGE HAS EXPIRED.
processing event {'type': 'check_if_cabbage_expired', 'time': 8.201130399525653}
CAB

processing event {'type': 'check_if_cabbage_expired', 'time': 222.3485898720452}
processing event {'type': 'cabbage_reorder', 'time': 222.44193715801396}
processing event {'type': 'cabbage_arrival', 'time': 222.44193715801396}
processing event {'type': 'customer_arrival', 'time': 222.98250163240814}
processing event {'type': 'customer_purchase', 'time': 222.98250163240814}
processing event {'type': 'customer_leaves_happy', 'time': 222.98250163240814}
processing event {'type': 'cabbage_reorder', 'time': 223.69419391064577}
processing event {'type': 'cabbage_arrival', 'time': 223.69419391064577}
processing event {'type': 'check_if_cabbage_expired', 'time': 224.4305343502517}
processing event {'type': 'customer_arrival', 'time': 224.45365352598665}
processing event {'type': 'customer_purchase', 'time': 224.45365352598665}
processing event {'type': 'customer_leaves_happy', 'time': 224.45365352598665}
processing event {'type': 'cabbage_reorder', 'time': 224.70493124907975}
processing event 